# Magniudes e erros com a tabela Truth do DP02

_Autores: Andreia Dourado, Bruno Moraes_

_Adaptado do README do repositório PhotErr: https://github.com/jfcrenshaw/photerr/tree/main_

__Descrição: Passo a passo para gerar as magnitudes observáveis e seus respectivos erros utilizando as magnitudes truth do DP02.__

### 1. Instalação do photoerr
__Executar:__ pip install photerr

### 2. Importando bibliotecas:

In [ ]:
from photerr import LsstErrorModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 3. Leitura do arquivo com os fluxos truth:

In [ ]:
catalog = pd.read_csv("/home/andreia.dourado/TCC/dp02/xmatch/data/xmatching_dp02_truth_object_0.0005.csv")

In [ ]:
catalog

### 4. Conversão dos fluxos para magnitudes:

In [ ]:
bands = ['u','g', 'r', 'i','z','y']
for f, filt in enumerate(bands):
    catalog[f'mag_{filt}_truth_dp02'] = -2.5 * np.log10(catalog[f'flux_{filt}_noMWdp01_test_truth']) + 31.4

### 5. Aplicação do LsstErrorModel


In [ ]:
catalog.columns

#### 5.1 Selecionando o ano de observação para adição dos erros e as colunas de magnitude truth: 

In [ ]:
errModel = LsstErrorModel(nYrObs=1,renameDict={"u": "mag_u_truth_dp02", "g": "mag_g_truth_dp02", "r": "mag_r_truth_dp02", "i": "mag_i_truth_dp02", "z": "mag_z_truth_dp02", "y": "mag_y_truth_dp02"})

#### 5.2 Gerando os erros e as magnitudes observáveis;

In [ ]:
catalog_with_errors = errModel(catalog, random_state=42)

In [ ]:
catalog_with_errors.columns

#### 5.3 Analisandos graficamento os erros gerados:

In [ ]:
def plot_errors(catalog, title='Errors', gridsize=[400, 200], bins='log', cmap='inferno', xlim=[20, 30], ylim=[0, 100], sigma=0, pop='n'):
    bands = ['u', 'g', 'r', 'i', 'z', 'y']
    fig, axes = plt.subplots(3, 2, figsize=[12, 16])
    fig.suptitle(title, fontsize=16)
    for ax, band in zip(axes.flatten(), bands):
        mag = np.array(catalog[f'mag_{band}_truth_dp02'])
        err = np.array(catalog[f'mag_{band}_truth_dp02_err'])
        sn = 1 / (10 ** (0.4 * err) - 1)
        ax.hexbin(mag, sn, gridsize=gridsize, cmap=cmap, bins=bins, mincnt=1)
        ax.set_ylabel("S/N", fontsize=14)
        ax.set_xlabel(f"mag {band}", fontsize=14)
        ax.set_ylim(ylim)
        ax.set_xlim(xlim)
        ax.axhline(5, color='black', label='5σ', linestyle='--')
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.legend(fontsize=12)
    fig.tight_layout(rect=[0, 0, 1, 0.97])

In [ ]:
plot_errors(catalog_with_errors)

#### 5.4 Salvando o arquivo:

In [ ]:
catalog_with_errors.to_csv("/home/andreia.dourado/TCC/dp02/truth/data/dp02_truth_lsst_error_model_Y5.csv")